# Inferential statistics II - Bootstrapping

## Introduction

In the previous frequentist mini-projects, you did frequentist calculations to perform inference from a sample of data. Such inference relies on theory largely developed from the 19th-Century onwards that is subject to certain assumptions or theoretical limits. These are fine if those assumptions hold for the particular case you're working on, and what you want to do has a known theoretical distribution (for example the mean of a sampling distribution that we looked at in the previous mini-project.)

In this mini-project, you'll use the same medical charge data you used in the frequentist inference mini-project, but this time you'll make inferences about the population using bootstrapping (ie. simulating repeated re-runs of an experiment.) If frequentism is about using assumptions and theoretical results to calculate what we expect to happen were an experiment to be run again and again and again, then bootstrapping is about using computing power to essentially re-run the sample draw again and again and again to see what actually happens.

## Prerequisites

While these exercises do not strictly depend on these concepts, we encourage you to complete the previous mini-projects before starting this one so that you can approach this assignment with a good understanding of frequentist concepts like:
* the _z_-statistic
* the _t_-statistic
* the difference and relationship between the two
* the Central Limit Theorem, its assumptions and consequences
* how to estimate the population mean and standard deviation from a sample
* the concept of a sampling distribution of a test statistic, particularly for the mean
* how to combine these concepts to calculate confidence intervals and p-values
* how those confidence intervals and p-values allow you to perform hypothesis (or A/B) tests

To complete mini-project, it's important that you first complete the bootstrap resources listed in this subunit, as they contain valuable information about how to calculate bootstrap replicates of summary statistics. Having an basic understanding of what confidence intervals and p-values are will also be helpful (we touch on them in this mini-project, but please speak to your mentor or conduct individual research if you'd like to learn more.) 

In [ ]:
import pandas as pd
import numpy as np
from numpy.random import seed
import matplotlib.pyplot as plt

## Medical charge data set

In [ ]:
med_charges = pd.read_csv('data/insurance2.csv')

In [ ]:
med_charges.head()

In the previous assignment, you used the frequentist approach to estimate the lower limit for the 95% confidence interval on the mean hospital charge. This approach relies on statistical theory that has been developed over the years and is also limited to statistics for which theoretical results on the sampling distribution exist. These results are remarkably useful and applicable much of the time and under a surprisingly wide range of conditions.

Having calculated the 95% lower confidence interval using frequentist theory in the previous exercise, you'll now use bootstrap inference to verify your calculations and check that you get consistent results without making the assumptions required before. After all, the distribution of charges really was very non-normal.

In [ ]:
charges_mean = np.mean(med_charges.charges)
charges_mean

__Q:__ Use bootstrap sampling to estimate the same 95% confidence interval lower limit as before.

__A:__

In [ ]:
np.random.seed(47)
N_rep = 10000

def bootstrap_replicate_1D(data, func):
    bs_sample = np.random.choice(data, len(data))
    return func(bs_sample)

#draw bootstrap replicates
def draws_bs_reps(data, func, size=1):
    bs_replicates = np.empty(size)
    for i in range(size):
        bs_replicates[i] = bootstrap_replicate_1D(data, func)
    return bs_replicates

#take 10000 bootstrap replicates of the mean
bs_replicates = draws_bs_reps(med_charges.charges, np.mean, N_rep)        

confidence_interval = np.percentile(bs_replicates, (5))
print('95% confidence interval =', confidence_interval)

If you performed 10000 replicates immediately after setting the random seed to 47, you should get the value 12724 here, which compares very well with the value 12725 obtained using the _t_-distribution confidence interval previously. It is a most pleasant result to see the predictions of classical frequentist theory match with results that are now possible through the number-crunching ability of computers.

Remember, in the previous mini-projects, we saw that there are two ways of performing a _t_-test from a sample, depending on whether we can assume the groups have equal variance or not. We can actually easily test this using the bootstrap approach!

__Q:__ Calculate the 95% confidence interval for the difference between the standard deviations of insurance and non-insurance claim charges (insured - non-insured). Calculate the differences over 10000 replicates. Plot the histogram of values and mark the locations of the percentiles. State the null and alternative hypothesis and comment on whether you would retain or reject the null hypothesis in this case and why.

__A:__

In [ ]:
#create two new DataFrame for insured and non_insured charges

insured = med_charges[med_charges.insuranceclaim == 1].charges

non_insured = med_charges[med_charges.insuranceclaim == 0].charges

In [ ]:
print('std for insured: ', np.std(insured, ddof=1))
print('std for non-insured: ', np.std(non_insured, ddof=1))

In [ ]:
print('difference of std', np.std(insured, ddof=1) - np.std(non_insured, ddof=1))

In [ ]:
#function to draw bootstrap replicates for difference betweeb std of two data sets
def draws_bootstrap_reps(data1, data2, func, size=1):
    bootstrap_replicates = np.empty(size)
    for i in range(size):
        std1 = bootstrap_replicate_1D(data1, func)
        std2 = bootstrap_replicate_1D(data2, func)
        bs_replicates[i] = std1 - std2
    return bs_replicates

In [ ]:
#draw bootstrap replicates for difference betweeb std of insured and non-insured
insured_non_insured_diff = draws_bootstrap_reps(insured, non_insured, np.std, N_rep)

In [ ]:
#95% confidence interval
confidence_interval = np.percentile(insured_non_insured_diff, [2.5, 97.5])
print('95% confidence interval =', confidence_interval)

In [ ]:
_ = plt.figure(figsize=[10,8])
_ = plt.hist(insured_non_insured_diff, bins=30, color='g', ec='k')
_ = plt.xlabel('charges')
_ = plt.ylabel('cases')
_ = plt.axvline(confidence_interval[0], color='r', linestyle='--')
_ = plt.axvline(confidence_interval[1], color='r', linestyle='--')

In [ ]:
mean_sample = np.mean(insured_non_insured_diff)
mean_sample

The null hypothesis is that the standard deviation of charges for insured and non_insured are the same.
As shown in the histogram, the difference of standard deviation from bootstrap simulation under the null hypothesis lies somewhere between 6000 and 9000.
Since the mean of the standard deviationof two data sets is not close to zero and the 95% confidence interval does not include zero, the hypothesis should be rejected.
By calculating the p-value, we can make sure to retain or reject the null hypothesis.

## Confidence interval and p-value

The confidence interval above is often a useful quantity to estimate. If we wish to limit our expected probability of making a Type I error (where we wrongly reject the null hypothesis, and it is, instead, true) to $\alpha$, the associated confidence interval is our estimate of the interval within which we expect the true population value to be found $100\times(1 - \alpha)$% of the time we do this test. In the above we performed bootstrap replicates to estimate the interval and reject the null hypothesis if this interval did not contain zero. You will sometimes see such an interval reported in the output of statistical functions.

The partner of the confidence interval is the p-value. The p-value and the confidence interval are linked through our choice of $\alpha$. The p-value tells us how likely it is, under the null hypothesis, to get an outcome at least as extreme as what was observed. If this fails to reach the level of our _pre-specified_ $\alpha$, we decide the null hypothesis is sufficiently unlikely to be true and thus reject it. To calculate this p-value via the bootstrap, we have to put ourselves in a position where we are simulating the null hypothesis being true and then calculate the fraction of times we observe a result at least as extreme as that actually observed.

Remember how, previously, you used the _t_-test to calculate the p-value for the observed difference between the means of insured and non-insured medical cases. We're now going to repeat this, this time using the bootstrap approach.

__Q:__ Perform a bootstrapped hypothesis test at the 5% significance level ($\alpha = 0.05$) to calculate the p-value of the observed difference between insurance and non-insurance charges, state your null and alternative hypotheses and whether you retain or reject the null hypothesis for the given significance level.

__A:__ The question is not clear. So, I calculate the p-value for two scenarios:
<p> 1) Null hypothesis: the insured and non_insured data sets have the same standard deviation.
<p> 2) Null hypothesis: the insured and non_insured data sets have the same mean.
    

Scenario 1: Null hypothesis is that insured and non_insured data sets have the same standard deviation.
<p>The test statistic is assumed as: np.std(insured) - np.std(non_insured)
<p>Compute the p-value. The achieved p-value is 0. Since the p-value is very low, the hypothesis should be rejected. 
<p>The alternative hypotheses is that there is difference between the standard deviation of insured and non_insured charges.

In [ ]:
# The difference of std of insured and non_insured:
diff_std = np.std(insured) - np.std(non_insured)
diff_std

In [ ]:
# Compute p-value: p
p = np.sum(insured_non_insured_diff >= diff_std) / len(insured_non_insured_diff)
print('p-value =', p)

In [ ]:
_ = plt.figure(figsize=[10,8])
_ = plt.hist(insured_non_insured_diff, bins=30, color='pink', ec='k')
_ = plt.xlabel('charges')
_ = plt.ylabel('cases')
_ = plt.axvline(diff_std, color='b', linestyle='--')
_ = plt.axvline(-diff_std, color='b', linestyle='--')

Scenario 2: Null hypothesis is that insured and non_insured data sets have the same mean.
<p>The test statistic is assumed as: np.mean(insured) - np.mean(non_insured)
<p>Compute the p-value. The achieved p-value is 0. Since the p-value is very low, the hypothesis should be rejected. 
<p>The alternative hypotheses is that there is difference between the mean of insured and non_insured charges.

In [ ]:
# The difference of mean of insured and non_insured:
diff_mean = np.mean(insured) - np.mean(non_insured)
diff_mean

In [ ]:
# Generate shifted arrays
insured_shifted = insured - np.mean(insured) + charges_mean
non_insured_shifted = non_insured - np.mean(non_insured) + charges_mean

In [ ]:
# Compute 10,000 bootstrap replicates from shifted arrays
shifted_bs_replicate = draws_bootstrap_reps(insured_shifted, non_insured_shifted, np.mean, N_rep)

In [ ]:
# Compute p-value: p
p = np.sum(shifted_bs_replicate >= diff_mean) / len(shifted_bs_replicate)
print('p-value =', p)

In [ ]:
#95% confidence interval
shifted_confidence_interval = np.percentile(shifted_bs_replicate, [2.5, 97.5])
print('95% confidence interval =', shifted_confidence_interval)

__Q:__ To put the above result in perspective, plot the histogram of your bootstrapped differences along with lines marking the locations of the observed difference. (Why would we plot more than one line, given that we only have one observed difference?)

__A:__

Two lines should be plotted since the result depends on how we subtract the data sets.

In [ ]:
_ = plt.figure(figsize=[10,8])
_ = plt.hist(shifted_bs_replicate, bins=30, color='yellow', ec='k')
_ = plt.xlabel('charges')
_ = plt.ylabel('cases')
_ = plt.axvline(diff_mean, color='r', linestyle='--')
_ = plt.axvline(-diff_mean, color='r', linestyle='--')

__Q:__ Compare your p-value above with that obtained using the _t_-test function in the previous assignment. Do you think you would want to try to perform enough bootstrap replicates to observe a random difference as large as that we did observe?

__A:__ No, the distribution is not even close to the observed difference and even if it is randomly observed, it does not change my conculusion about rejection of null hypothesis.

__Q:__ Consider the two variants of the _t_-test we performed in the previous assignment. Which one would you use now?

__A:__ p-value gives us better results to decide to retain or reject a hypothesis. Deciding from histogram may not be accurate.

__Q:__ If, instead of being asked whether the means of two groups were different, you were working with an ad-recommender team who wanted to release a new recommendation algorithm, and you were looking at click-through rate both for the current algorithm (call it A) and from trials of their new algorithm (call it B), would you perform a two-sided test as above? What would be your null and alternative hypothesis and what would be the real-world consequence of rejecting the null hypothesis?

__A:__ Yes, I would perform a two-sided test for this experiment. However, I think permutaion test would be better option. 
<p>The null hypothesis: The click-through rate is not affected by algorithm redesigning.
<p>The alternative hypothesis: Redesigning the algorithm has a great impact on click-through rate.
<p> If the null hypothesis is rejected, new algorith should be develop to increase the click-through rate.

# Learning outcomes

You've previously applied frequentist methods to calculate confidence intervals, p-values, and perform hypothesis tests. Frequentist methods use theoretical results to calculate what we expect would happen if experiments were to be run again and again and again. Now you've seen how you can do the same things using the bootstrap approach, which does not rely on such theory, and attendant assumptions, but instead literally does run experiments again and again and again.

In these exercises, you have:
* calculated the same confidence interval lower limit as you did previously
* tested the assumption that the variances of the two groups (insured vs. non-insured) were equal - something a bit harder to do using the frequentist method because of the nature of the sampling distribution for variance
* calculated the p-value for the difference between the means of the two groups and compared with the result obtained using the previous frequentist approach

You are now well equipped to apply the bootstrap approach to a wide variety of problems. Just think about what conditions you wish to recreate in your simulated reruns.